In [ ]:
import random, os, itertools, sys, warnings, logging, json, multiprocessing, time, glob

# FOR GPU SELECTION
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "3" # -1 = not using gpu
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

logging.disable(logging.WARNING)
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import ngboost as ngb

from utils.evaluator import compute_anomaly_scores, compute_new_metrics
from utils.experiment import core_seed, running_seeds, save_model_configs, save_results
from utils.experiment import data_loaders, seq_lengths, strides, n_epochs, batch_size, clear_memory

from tensorflow import keras
from tensorflow.keras import layers, optimizers, callbacks, Input, Model
from tensorflow.keras.losses import MSE, MAE, logcosh 

from PASTA.graph_builder import build_graph
from PASTA.search_space import SearchSpace

from tqdm.notebook import tqdm

In [ ]:
data_name = 'psm' # ['tods', 'asd', 'psm'] for SWaT, please request the dataset first
budget = 100
z_dim = 32 # default value

In [ ]:
# THESE LINES ARE FOR REPRODUCIBILITY
random.seed(core_seed)
np.random.seed(core_seed)
tf.random.set_seed(core_seed)

In [ ]:
# Data Loaders 
seq_length, stride = seq_lengths[data_name], strides[data_name]
data = data_loaders[data_name](seq_length=seq_length, stride=stride)

In [ ]:
def valid_topK(idx): # arch_matrices, graph_configs, arch_connections
    
    scores = []     
    main_configs, layer_configs = graph_configs[idx]
    reverse_output = main_configs["reverse_output"]

    selected_idx = list(range(len(data['x_train']))) # dataset index in a benchmark
        
    for i in tqdm(selected_idx):
        x_train, x_valid, x_test = data['x_train'][i], data['x_valid'][i], data['x_test'][i]
        y_valid, y_test = data['y_valid'][i], data['y_test'][i]
        y_segment_valid, y_segment_test = data['y_segment_valid'][i], data['y_segment_test'][i]

        start_time = time.time()
        model = build_graph(x_train.shape, graph_configs[idx], arch_connections[idx])
        build_time = time.time() - start_time
        print(f'graph built: {build_time:.2f} seconds.')

        if reverse_output:
            x_train_reverse = np.flip(x_train, axis=1)
            x_valid_reverse = np.flip(x_valid, axis=1)
            x_test_reverse = np.flip(x_test, axis=1)

            # Prepare the training dataset.
            train_dataset = tf.data.Dataset.from_tensor_slices((x_train, x_train_reverse))
            train_dataset = train_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

            # Prepare the validation dataset.
            valid_dataset = tf.data.Dataset.from_tensor_slices((x_valid, x_valid_reverse))
            valid_dataset = valid_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
        else:
            # Prepare the training dataset.
            train_dataset = tf.data.Dataset.from_tensor_slices((x_train, x_train))
            train_dataset = train_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

            # Prepare the validation dataset.
            valid_dataset = tf.data.Dataset.from_tensor_slices((x_valid, x_valid))
            valid_dataset = valid_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

        patience = 5
        losses = {
            'mse': tf.keras.losses.MSE,
            'mae': tf.keras.losses.MAE,
            'logcosh': tf.keras.losses.logcosh
        }
        optimizer = optimizers.Adam()
        loss_fn = main_configs["loss_fn"]
        es = callbacks.EarlyStopping(monitor="val_loss", patience=patience, mode="min", restore_best_weights=True)

        start_time = time.time()
        model.compile(loss=loss_fn, optimizer=optimizer)
        logs = model.fit(train_dataset, validation_data=valid_dataset, epochs=n_epochs, callbacks=[es], verbose=2)
        train_time = time.time() - start_time
        print(f'Train Time: {train_time:.2f}s')

        if reverse_output:
            train_pred = [np.flip(rec, axis=1) for rec in model.predict(x_train)]
            valid_pred = [np.flip(rec, axis=1) for rec in model.predict(x_valid)]
        else:
            train_pred = model.predict(x_train)
            valid_pred = model.predict(x_valid)

        train_pred = np.array(train_pred)
        valid_pred = np.array(valid_pred)  

        valid_rec = compute_anomaly_scores(x_valid, valid_pred, scoring=main_configs["scoring"], x_val = x_train, rec_val = train_pred)
        valid_scores = compute_new_metrics(valid_rec, y_valid, stride=stride)

        scores.append(max(valid_scores['eTaF1'])) # search by best-F1

    return {'perf': np.average(scores), 'idx': idx}

def run_full_train(rid:int, seed:int, arch_matrices, graph_configs, arch_connections):
    
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
        try:
            tf.config.set_logical_device_configuration(
                gpus[0],
                [tf.config.LogicalDeviceConfiguration(memory_limit=10000)])
            logical_gpus = tf.config.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
        except RuntimeError as e:
            # Virtual devices must be set before GPUs have been initialized
            print(e)    
    
    tf.keras.backend.clear_session()
    
    # setting for each run
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)    
    
    main_configs, layer_configs = graph_configs
    reverse_output = main_configs["reverse_output"]   
    
    selected_idx = list(range(len(data['x_train']))) # dataset index in a benchmark
        
    for i in tqdm(selected_idx):
        x_train, x_valid, x_test = data['x_train'][i], data['x_valid'][i], data['x_test'][i]
        y_valid, y_test = data['y_valid'][i], data['y_test'][i]
        y_segment_valid, y_segment_test = data['y_segment_valid'][i], data['y_segment_test'][i]
        
        start_time = time.time()
        model = build_graph(x_train.shape, graph_configs, arch_connections)
        build_time = time.time() - start_time
        print(f'graph built: {build_time:.2f} seconds.')
        
        if reverse_output:
            x_train_reverse = np.flip(x_train, axis=1)
            x_valid_reverse = np.flip(x_valid, axis=1)
            x_test_reverse = np.flip(x_test, axis=1)
            
            # Prepare the training dataset.
            train_dataset = tf.data.Dataset.from_tensor_slices((x_train, x_train_reverse))
            train_dataset = train_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

            # Prepare the validation dataset.
            valid_dataset = tf.data.Dataset.from_tensor_slices((x_valid, x_valid_reverse))
            valid_dataset = valid_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
        else:
            # Prepare the training dataset.
            train_dataset = tf.data.Dataset.from_tensor_slices((x_train, x_train))
            train_dataset = train_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

            # Prepare the validation dataset.
            valid_dataset = tf.data.Dataset.from_tensor_slices((x_valid, x_valid))
            valid_dataset = valid_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
        
        patience = 5
        losses = {
            'mse': tf.keras.losses.MSE,
            'mae': tf.keras.losses.MAE,
            'logcosh': tf.keras.losses.logcosh
        }
        optimizer = optimizers.Adam()
        loss_fn = main_configs["loss_fn"]
        es = callbacks.EarlyStopping(monitor="val_loss", patience=patience, mode="min", restore_best_weights=True)
        
        start_time = time.time()
        model.compile(loss=loss_fn, optimizer=optimizer)
        logs = model.fit(train_dataset, validation_data=valid_dataset, epochs=n_epochs, callbacks=[es], verbose=2)
        train_time = time.time() - start_time
        print(f'Train Time: {train_time:.2f}s')
        
        if reverse_output:
            train_pred = [np.flip(rec, axis=1) for rec in model.predict(x_train)]
            valid_pred = [np.flip(rec, axis=1) for rec in model.predict(x_valid)]
            test_pred = [np.flip(rec, axis=1) for rec in model.predict(x_test)]
            train_errors = logs.history['loss'] # model.evaluate(x_train, x_train_reverse)
            valid_errors = logs.history['val_loss'] # model.evaluate(x_valid, x_valid_reverse)
            test_errors = model.evaluate(x_test, x_test_reverse, verbose=0)            
        else:
            train_pred = model.predict(x_train)
            valid_pred = model.predict(x_valid)
            test_pred = model.predict(x_test)
            train_errors = logs.history['loss'] # model.evaluate(x_train, x_train)
            valid_errors = logs.history['val_loss'] # model.evaluate(x_valid, x_valid)
            test_errors = model.evaluate(x_test, x_test, verbose=0)     
            
        train_pred = np.array(train_pred)
        valid_pred = np.array(valid_pred)  
        test_pred = np.array(test_pred)
        
        start_time = time.time()
        test_rec = compute_anomaly_scores(x_test, test_pred, scoring=main_configs["scoring"], x_val = x_valid, rec_val = valid_pred)
        test_scores = compute_new_metrics(test_rec, y_test, stride=stride)
        test_time = time.time() - start_time
        print(f'Test Time: {test_time:.2f}s, {max(test_scores["eTaF1"])}') # selecting best F1

In [ ]:
def predict_z(X, return_dict):
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
        try:
            tf.config.set_logical_device_configuration(gpus[0], [tf.config.LogicalDeviceConfiguration(memory_limit=10000)])
            logical_gpus = tf.config.list_logical_devices('GPU')
        except RuntimeError as e:
            # Virtual devices must be set before GPUs have been initialized
            print(e)    
    
    tf.keras.backend.clear_session()
    
    model = tf.keras.models.load_model(f'results/pretrained_models/PASTA_CTAE/model_{seq_length}_{z_dim}_all/')
    encoder = tf.keras.Model(inputs=model.inputs, outputs=model.get_layer('encoder_output').output)
    
    return_dict[0] = np.array(encoder.predict(X))

In [ ]:
# search configuration
# N = evaluation pool size, (randomly select) subset of top-k, sieze of top-k architecture subset
N, Ni, TopK = int(1e3), 20, 100

In [ ]:
used_budget = 0
start_time = time.time()
f_names = glob.glob(f'results/architectures/{data_name.upper()}/Full/*.npy') + glob.glob(f'results/architectures/{data_name.upper()}/Reduced/*.npy')

In [ ]:
y_true_f1 = []
x_setting_test = []
x_network_test = []
x_temp_test = []
x_encoder_test = []
x_decoder_test = []

for f_name in tqdm(f_names):
    test_arch = np.load(f_name, allow_pickle=True).item()
    xs_test = test_arch['onehot']
    xc_test = test_arch['connection']

    x_setting_test.append(xs_test[0])
    x_network_test.append(xs_test[1])
    x_temp_test.append(xs_test[2])
    x_encoder_test.append(xc_test[0])
    x_decoder_test.append(xc_test[1])

    y_true_f1.append(np.average(test_arch['scores']['valid']['eTaF1']))

x_test = [np.array(x_setting_test), np.array(x_network_test), np.array(x_temp_test), np.array(x_encoder_test), np.array(x_decoder_test)]
y = np.array(y_true_f1)

# to enable clearing GPU memory usage after prediction
manager = multiprocessing.Manager()
return_dict = manager.dict()

p = multiprocessing.Process(target=predict_z, args=(x_test, return_dict,))
p.start()
p.join()

Z = return_dict.values()[0]

In [ ]:
N0 = int(budget * 0.50) # use 50% of budget to initialize the performace predictor
predictor = ngb.NGBRegressor(random_state=core_seed, verbose=0)
train_idx = np.random.choice(len(Z), size=N0, replace=False)
Z0, y0 = Z[train_idx], y[train_idx]
predictor.fit(Z0, y0)
used_budget += N0
print(f'{used_budget}/{budget}')

In [ ]:
# search on evaluation pool
search_space = SearchSpace()
search_space.build_search_space(N)
arch_matrices = search_space.get_random_architectures(N, with_adj = True)
graph_configs = search_space.get_architecture_configs(arch_matrices["onehot"])
arch_connections =  []
for config in tqdm(graph_configs):
    arch_connections.append(search_space.get_architecture_connections(config, seq_length))    
xs_test = arch_matrices['onehot']
xc_test = arch_connections

x_setting_test = []
x_network_test = []
x_temp_test = []
x_encoder_test = []
x_decoder_test = []

for i in range(N):
    x_setting_test.append(xs_test[i][0])
    x_network_test.append(xs_test[i][1])
    x_temp_test.append(xs_test[i][2])
    x_encoder_test.append(xc_test[i][0])
    x_decoder_test.append(xc_test[i][1])

x_test = [np.array(x_setting_test), np.array(x_network_test), np.array(x_temp_test), np.array(x_encoder_test), np.array(x_decoder_test)]

manager = multiprocessing.Manager()
return_dict = manager.dict()

p = multiprocessing.Process(target=predict_z, args=(x_test, return_dict,))
p.start()
p.join()

Z = return_dict.values()[0]
Z_Ni = Z0
y_Ni = y0
selected_idx = []
while used_budget < budget:
    y_pred = predictor.predict(Z).ravel()
    topK_idx = np.argpartition(y_pred, -TopK)[-TopK:]
    topK_Ni_idx = np.random.choice(topK_idx, size=Ni, replace=False)

    Z_Ni = np.concatenate([Z[topK_Ni_idx], Z_Ni], axis=0)
    y_Ni = np.concatenate([y_pred[topK_Ni_idx], y_Ni], axis=0)
    predictor.fit(Z_Ni, y_Ni) # P_i+1
    Z = np.delete(Z, topK_Ni_idx, axis=0) # exclude those obsereved!

    used_budget += Ni
    print(f'{used_budget}/{budget}')

In [ ]:
K = 3 # number of predicted top-k architectures for final validation
y_pred = predictor.predict(Z).ravel()
topK_idx = np.argpartition(y_pred, -K)[-K:] # get predicted best arch index
print(y_pred[topK_idx])
search_time = time.time() - start_time
print(f'Search Time: {search_time}')

with multiprocessing.Pool(1) as pool:
    results = pool.map(valid_topK, topK_idx)

In [ ]:
best_idx = None
best_valid = -np.inf
for result in results:
    if result['perf'] > best_valid:
        best_idx = result['idx']
        best_valid = result['perf']

print(f'Best Valid: {best_valid}')

# full train the best validated architecture
for rid, seed in enumerate(running_seeds(1)):
    p = multiprocessing.Process(target=run_full_train, args=(rid, seed, arch_matrices["onehot"][best_idx], graph_configs[best_idx], arch_connections[best_idx],))
    p.start()
    p.join()